In [9]:
from typing import TextIO,Dict
import random
from typing import Optional
from tqdm import tqdm


In [10]:
class TreeNode():
    _prefix_branches:Dict[str,"TreeNode"]
    tokens:Dict[str,int]

    def __init__(self) -> None:
        self._prefix_branches={}
        self.tokens={}


    def __repr__(self) -> str:
        return f"{self.tokens}"

    def add_prefix_branch(self,token:str)->"TreeNode":
        if token in self._prefix_branches:
            raise RuntimeError(f"Token exists already in prefix {token}")
        self._prefix_branches[token]=TreeNode()
        self.tokens[token]=1
        return self._prefix_branches.get(token)

    def get_prefix_branch(self,token:str)->Optional["TreeNode"]:
        return self._prefix_branches.get(token)

    def num_branches(self)->int:
        return len(self._prefix_branches)

    def increment_token(self,token:str,by:int=1)->None:
        if token in self.tokens:
            self.tokens[token]=self.tokens[token]+by

class PrefixTree():
    _root:TreeNode

    def __init__(self) -> None:
        self._root=TreeNode()

    def add_value(self,value:str)->None:
        current:TreeNode=self._root
        for char in value:
            current.increment_token(char)
            current = self._traverse_single_level(current,char,add_if_missing=True)
            assert current is not None

    def get_prefix(self,value:str)->Optional[Dict[str,int]]:
        current:TreeNode=self._root
        for char in value:
            current = self._traverse_single_level(current,char,add_if_missing=False)
            if not current:
                return None
        return current.tokens


    def _traverse_single_level(self,node:TreeNode,char:str,add_if_missing:bool)->Optional[TreeNode]:
        assert len(char)==1

        retrun_node = node.get_prefix_branch(char)
        if retrun_node is None and add_if_missing:
            return node.add_prefix_branch(char)

        return retrun_node


    def __repr__(self)->str:
        return self._print_node(self._root,"")


    def _print_node(self,node:TreeNode,substr:str)->str:
        if node.num_branches()==0:
            return ""
        current = f"{substr}{node}\n"
        for sub_b in node._prefix_branches.values():
            current+= self._print_node(sub_b,f"{substr}--")
        return current





In [12]:
#now develop some conditional memory
#a small class will help us with book keeping
class MarkovChain():

    def __init__(self,window_size:int=1,eod:str="\u0000") -> None:
        assert window_size>0
        self._window_size=window_size
        self._memory=PrefixTree()
        self._current_buffer=[str(0x0) for _ in range(window_size+1)] #bootstrap with null chars
        self._generate_buffer=""
        self._eod=eod #end of document special token


    def update(self,token:str)->None:
        assert len(token)==1
        self._current_buffer.append(token)
        self._current_buffer.pop(0)
        for i in range(1,self._window_size+2):
            self.update_item("".join(self._current_buffer[-i:]))


    def update_item(self,string:str)->None:
        self._memory.add_value(string)

    def _next_char(self)->str:
        prefixes = self._memory.get_prefix(self._generate_buffer)


        if prefixes is None:
            raise ValueError("i've never seen this before :(")

        population = []
        counts=[]

        for string,count in prefixes.items():
            population.append(string)
            counts.append(count)

        return random.choices(population=population,weights=counts,k=1)[0][-1:]

    def generate_string(self,init:str,n=500)->str:
        self._generate_buffer=init[-self._window_size:]

        for _ in range(n):
            next_char = self._next_char()
            if next_char==self._eod:
                break
            self._generate_buffer = (self._generate_buffer+next_char)[-self._window_size:]
            yield next_char

    def train(self,tokenizer)->None:
        for token in tqdm(tokenizer):
            self.update(token)


In [13]:
EOD="\u0000"
mc=MarkovChain(window_size=10,eod=EOD)

def naive_tokenizer(file:TextIO,chunk_size:int=1024)->str:
    """assume the file is one huge stream of text"""
    while True:
        str_chunk = file.read(chunk_size)
        if not str_chunk:
            yield EOD
            break
        for char in str_chunk:
            yield char

def wiki_tokenizer(file:TextIO)->str:
    """tokenizer for the wiki text dataset format"""
    for line in file.readlines():
        line = line.strip()
        if line.startswith('"text": '):
            for char in line[8:-1]:
                yield char
            yield EOD


with open("data/shakespeare.txt", mode="r", encoding="utf-8") as file_obj:
    mc.train(naive_tokenizer(file_obj))

with open("data/enwiki20201020/0aaf9aae-5557-466c-8c50-13170a3cbec8.json", mode="r", encoding="utf-8") as file_obj:
    mc.train(wiki_tokenizer(file_obj))


11606603it [11:01, 17558.73it/s]


In [22]:
init_string = "inseminate"
print(f'{init_string}{"".join([char for char in  mc.generate_string(init_string,100000)])}')

inseminated females found a job at the University alumni Category:Retail markets are called \"Seelenblindheit\" (mind-blindness) Heinrich Himmler became earl of Southampton do we shift our scene must find us
And, which focuses on how the FBI secretly instructs me.

[_Exit._]

SCENE II. Friar Lawrence met themselves, and less than 10 minutes' walk away from the area was concerned, although Yū tries to complete master; but to jig off a tune at the time, thou art not fish; if thou wilt, and
those that it was the first lunar month, the court.

RUGBY.
Forbear till this afternoon. {| class=\"wikitable\" width=\"60\"|G !width=\"60\"|L !width=\"250px\"|Director !style=\"background- color:;\"| |Louis Feuvrier |PS dissident writer against Coldplay performed a medley of the canyon. Cedar-bark sheathing, wood- shingled roofs, and rustic stone masonry building on Greenville. In May 2013, Tropicana sold the following the picture. Alas the day, three thousand hearts.
But I must make conceived and led